# Script to create graph and reactions from input matrix


In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import os 


In [2]:
state_vars_per_gene = ["is_bursting", "unspliced_mRNA", "spliced_mRNA", "protein",
                      "k_on_adjusted", "total_mRNA", 
                      "mRNA_ever_produced", "protein_ever_produced"]
#Define indices for each state var
state_var_indices = {var: i for i, var in enumerate(state_vars_per_gene)}

In [3]:
#To read the input matrix
def read_input_matrix(path_to_matrix):
    """
    Reads the input matrix from the specified file path and counts number of genes.
    
    Parameters:
    path_to_matrix (str): The file path to the input matrix.
    
    Returns:
    np.ndarray: The input matrix as a NumPy array.
    """
    matrix = np.loadtxt(path_to_matrix, dtype='i', delimiter=',')
    return matrix.shape[0], matrix

In [4]:
#Read the parameters
def assign_parameters_to_genes(csv_path, rows=None, n_random=3):
    """
    Select specific rows or random rows from parameter CSV and map to genes.
    
    Args:
        csv_path (str): Path to CSV file
        rows (list, optional): Specific row indices to select. If None, selects random rows
        n_random (int): Number of random rows if rows=None
    
    Returns:
        tuple: (param_dict, row_mapping)
            param_dict: {gene_id: {param_name: value}}
            row_mapping: {gene_id: row_number}
    """
    df = pd.read_csv(csv_path, index_col=0)

    # If no specific rows are provided, select random rows to assugn to genes
    if rows is None:
        rows = np.random.choice(df.index, size=min(n_random, len(df)), replace=True)
    
    param_dict = {}
    row_mapping = {}
    
    for i, row in enumerate(rows):
        if row not in df.index:
            raise ValueError(f"Row {row} not found in the DataFrame.")
        gene_id = f"gene_{i+1}"
        param_dict[gene_id] = df.loc[row].to_dict()
        row_mapping[gene_id] = row
    # Create a matrix where genes are rows and parameters are columns
    param_matrix = pd.DataFrame.from_dict(param_dict, orient='index')

    #Add additional calculations for degradation, transcription rate, splicing rate, etc.
    param_matrix['mRNA_degradation_rate'] = np.log(2)/param_matrix['mrna_half_life']
    param_matrix['protein_degradation_rate'] = np.log(2)/param_matrix['protein_half_life']
    param_matrix['transcription_rate'] = param_matrix['burst_size'] * param_matrix['k_off']

    #Remove old columns that are not needed
    param_matrix.drop(columns=['mrna_half_life', 'protein_half_life', 'burst_size'], inplace=True, errors='ignore')

    return param_matrix, row_mapping

In [5]:
def save_complete_snapshot(state_array, state_var_indices, timestep=None):
    """
    Reshape from (n_genes, n_state_vars, n_cells) to (n_cells, n_genes × n_state_vars) DataFrame
    """
    n_genes, n_state_vars, n_cells = state_array.shape

    # Prepare data columns
    columns = []
    data = []

    for gene_idx in range(n_genes):
        for var_name, var_idx in state_var_indices.items():
            col_name = f"gene_{gene_idx+1}_{var_name}"
            columns.append(col_name)
            data.append(state_array[gene_idx, var_idx, :])  # shape (n_cells,)

    # Stack and transpose to (n_cells, n_features)
    df_data = np.stack(data, axis=1)  # shape: (n_cells, n_genes × n_state_vars)
    df = pd.DataFrame(df_data, columns=columns)
    df["cell_id"] = df.index # Add cell IDs as a column
    # Add metadata
    if timestep is not None:
        df["timestep"] = timestep
    return df

In [6]:
from scipy.optimize import fsolve
import numpy as np
import pandas as pd

def get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params=None, target_hill=0.5):
    """
    Calculate steady-state protein levels for all genes, to use as EC50 values.
    Supports per-gene max_effect and Hill coefficient n.
    Uses additive regulatory effects: activators increase k_on, repressors increase k_off.
    """
    if global_params is None:
        global_params = {
            "splicing_rate": np.log(2)/(7/60),
            "max_effect": 16,
            "n": 2
        }

    splicing_rate = global_params["splicing_rate"]
    n_genes = param_matrix.shape[0]

    # Parameters
    k_on = param_matrix["k_on"].values
    k_off = param_matrix["k_off"].values
    txn_rate = param_matrix["transcription_rate"].values
    m_deg = param_matrix["mRNA_degradation_rate"].values
    p_prod = param_matrix["protein_production_rate"].values
    p_deg = param_matrix["protein_degradation_rate"].values

    # Fix negative k_off
    k_off = np.where(k_off < 0, 1e-6, k_off)

    # Per-gene max_effect
    max_effect = global_params["max_effect"]
    if np.isscalar(max_effect):
        max_effect = np.full(n_genes, max_effect)
    else:
        max_effect = np.asarray(max_effect)

    # Per-gene n (Hill coefficient)
    n_param = global_params["n"]
    if np.isscalar(n_param):
        n_param = np.full(n_genes, n_param)
    else:
        n_param = np.asarray(n_param)

    def system(P):
        eqs = []
        for i in range(n_genes):
            regulators = np.where(interaction_matrix[:, i] != 0)[0]
            act_sum = 0.0
            rep_sum = 0.0

            for j in regulators:
                weight = interaction_matrix[j, i]
                n_i = n_param[i]
                hill_response = (P[j] ** n_i) / (P[j] ** n_i + 1.0)

                if weight > 0:
                    act_sum += weight * hill_response
                elif weight < 0:
                    rep_sum += abs(weight) * hill_response

            # Adjusted k_on and k_off
            k_on_eff = k_on[i] + max_effect[i] * act_sum
            k_off_eff = k_off[i] + max_effect[i] * rep_sum
            k_on_eff = np.clip(k_on_eff, 1e-10, None)
            k_off_eff = np.clip(k_off_eff, 1e-10, None)

            # Compute protein level
            burst_prob = k_on_eff / (k_on_eff + k_off_eff)
            unspliced = txn_rate[i] * burst_prob / (m_deg[i] + splicing_rate)
            spliced = unspliced * splicing_rate / m_deg[i]
            protein = spliced * p_prod[i] / p_deg[i]

            eqs.append(protein - P[i])
        return eqs

    # Initial guess: unregulated steady state
    guess = []
    for i in range(n_genes):
        b = k_on[i] / (k_on[i] + k_off[i])
        u = txn_rate[i] * b / (m_deg[i] + splicing_rate)
        s = u * splicing_rate / m_deg[i]
        p = s * p_prod[i] / p_deg[i]
        guess.append(p)

    try:
        result = fsolve(system, guess)
        return np.maximum(result, 0.1)
    except Exception as e:
        print(f"⚠️ Failed to solve steady state: {e}")
        return np.array(guess)

In [7]:
def create_EC50_vector(param_matrix, interaction_matrix, global_params=None):
    """
    Create EC50 matrix where EC50[i,j] = steady-state protein level of regulator j
    when it regulates target i.
    
    Args:
        param_matrix: DataFrame with parameter values
        interaction_matrix: regulatory interaction matrix
        global_params: global parameters
    
    Returns:
        np.ndarray: EC50 matrix (shape: n_genes, n_genes)
    """
    # Get steady-state protein levels
    steady_state_protein = get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params)
    return np.maximum(steady_state_protein, 1e-12)  # Avoid 0 in denominator

In [8]:
def get_new_state(state_array, param_matrix, interaction_matrix, state_var_indices, n_genes, n_state_vars, num_cells,
                                   EC50_matrix=None, global_params=None, t=1/60, ):
    """
    Pure 3D matrix operations - everything happens simultaneously across all genes and cells.
    
    Args:
        state_array: shape (n_genes, n_state_vars, num_cells)
        param_matrix: DataFrame with genes as rows, parameters as columns
        interaction_matrix: shape (n_genes, n_genes) - regulatory interactions
        state_var_indices: dict mapping state variable names to indices
        global_params: dict of global parameters
        t: time step
    
    Returns:
        np.ndarray: updated state_array
    """

    if global_params is None:
        global_params = {"splicing_rate": np.log(2)/(7/60), "max_effect": 16, "n": 2}
    
    assert state_array.shape == (n_genes, n_state_vars, num_cells)
    new_state = state_array.copy()
    
    # Short aliases
    s = state_var_indices
    
    # Global parameters
    splicing_rate = global_params.get("splicing_rate", np.log(2)/(7/60))
    max_effect = global_params.get("max_effect", 16)
    n = global_params.get("n", 2)
    
    # === STEP 1: DEGRADATION - ALL GENES & CELLS AT ONCE ===
    
    # Create degradation rate arrays from DataFrame columns
    mrna_deg_rates = param_matrix['mRNA_degradation_rate'].values.reshape(n_genes, 1, 1)
    protein_deg_rates = param_matrix['protein_degradation_rate'].values.reshape(n_genes, 1, 1)
    
    # Apply degradation to entire state array slices - checked the dimensions
    unspliced_degraded = np.random.poisson(
        mrna_deg_rates * new_state[:, s["unspliced_mRNA"]:s["unspliced_mRNA"]+1, :] * t
    )
    spliced_degraded = np.random.poisson(
        mrna_deg_rates * new_state[:, s["spliced_mRNA"]:s["spliced_mRNA"]+1, :] * t
    )
    protein_degraded = np.random.poisson(
        protein_deg_rates * new_state[:, s["protein"]:s["protein"]+1, :] * t
    )
    
    # Update entire slices at once
    new_state[:, s["unspliced_mRNA"], :] = np.maximum(
        new_state[:, s["unspliced_mRNA"], :] - unspliced_degraded.squeeze(1), 0
    )
    new_state[:, s["spliced_mRNA"], :] = np.maximum(
        new_state[:, s["spliced_mRNA"], :] - spliced_degraded.squeeze(1), 0
    )
    new_state[:, s["protein"], :] = np.maximum(
        new_state[:, s["protein"], :] - protein_degraded.squeeze(1), 0
    )
    
    # === STEP 2: TRANSCRIPTION - ALL GENES & CELLS AT ONCE ===
    
    # Transcription rates from DataFrame column
    txn_rates = param_matrix['transcription_rate'].values.reshape(n_genes, 1)
    
    # Generate new mRNA for ALL genes and cells simultaneously
    new_mRNA = txn_rates * new_state[:, s["is_bursting"], :] * t
    
    # Add to unspliced mRNA pool - entire matrix operation
    new_state[:, s["unspliced_mRNA"], :] += new_mRNA
    new_state[:, s["mRNA_ever_produced"], :] += new_mRNA
    
    # === STEP 3: SPLICING - ALL GENES & CELLS AT ONCE ===
    
    # Splicing: unspliced → spliced for entire matrix
    spliced_amount = splicing_rate * new_state[:, s["unspliced_mRNA"], :] * t
    
    new_state[:, s["unspliced_mRNA"], :] -= spliced_amount
    new_state[:, s["spliced_mRNA"], :] += spliced_amount
    
    # === STEP 4: TRANSLATION - ALL GENES & CELLS AT ONCE ===
    
    # Protein production rates from DataFrame column
    protein_prod_rates = param_matrix['protein_production_rate'].values.reshape(n_genes, 1)
    
    # Create protein for ALL genes and cells simultaneously
    new_protein = protein_prod_rates * new_state[:, s["spliced_mRNA"], :] * t
    
    new_state[:, s["protein"], :] += new_protein
    new_state[:, s["protein_ever_produced"], :] += new_protein
    
    # Update total mRNA - entire matrix operation
    new_state[:, s["total_mRNA"], :] = (new_state[:, s["unspliced_mRNA"], :] + 
                                        new_state[:, s["spliced_mRNA"], :])
    
    # === STEP 5: K_ON ADJUSTMENT ===
    
    # === Additive interaction === #
    if interaction_matrix is not None and np.any(interaction_matrix != 0):
        protein_levels = new_state[:, s["protein"], :]  # (n_genes, num_cells)

        # Per-gene Hill coefficient
        if np.isscalar(n):
            n_vector = np.full((n_genes, 1), n)
        else:
            n_vector = np.asarray(n).reshape(n_genes, 1)  # shape: (n_genes, 1)

        # Per-gene EC50
        if EC50_matrix is not None and EC50_matrix.ndim == 1:
            EC50_vector = EC50_matrix.reshape(n_genes, 1)  # shape: (n_genes, 1)
        else:
            raise ValueError("EC50 input must be a 1D array when using vectorized logic.")

        # Hill response: shape (n_genes, num_cells)
        numerator = protein_levels ** n_vector
        denominator = EC50_vector ** n_vector + protein_levels ** n_vector
        hill_responses = numerator / (denominator + 1e-10)

        # Split interaction matrix
        activator_matrix = np.where(interaction_matrix > 0, interaction_matrix, 0)
        repressor_matrix = np.where(interaction_matrix < 0, -interaction_matrix, 0)  # flip sign for additive effect

        # Regulatory effects: shape (n_genes, num_cells)
        activation_effects = activator_matrix.T @ hill_responses
        repression_effects  = repressor_matrix.T @ hill_responses

        # Per-gene max_effect
        if np.isscalar(max_effect):
            max_effect_vector = np.full((n_genes, 1), max_effect)
        else:
            max_effect_vector = np.asarray(max_effect).reshape(n_genes, 1)

        # Adjusted k_on and k_off
        base_k_on  = param_matrix['k_on'].values.reshape(n_genes, 1)
        base_k_off = param_matrix['k_off'].values.reshape(n_genes, 1)
        k_on_adjusted  = base_k_on  + max_effect_vector * activation_effects
        k_off_adjusted = base_k_off + max_effect_vector * repression_effects

        # Ensure positivity
        k_on_adjusted  = np.maximum(k_on_adjusted, 1e-10)
        k_off_adjusted = np.maximum(k_off_adjusted, 1e-10)
    else:
        # No regulation: use base k_on/k_off tiled across cells
        k_on_adjusted  = np.tile(param_matrix['k_on'].values.reshape(n_genes, 1), (1, num_cells))
        k_off_adjusted = np.tile(param_matrix['k_off'].values.reshape(n_genes, 1), (1, num_cells))

    
    # Store in state array
    new_state[:, s["k_on_adjusted"], :] = k_on_adjusted
    new_state[:, s["k_off_adjusted"], :] = k_off_adjusted
    

    # # === Multiplicative interaction === #
    # if interaction_matrix is not None and np.any(interaction_matrix != 0):
    #     protein_levels = new_state[:, s["protein"], :]  # shape: (n_genes, num_cells)

    #     if EC50_matrix is not None and EC50_matrix.ndim == 1:
    #         EC50_vector = EC50_matrix.reshape(n_genes, 1)
    #         hill_responses = (protein_levels ** n) / (EC50_vector ** n + protein_levels ** n)

    #         # Clip to avoid log(0)
    #         hill_clipped = np.clip(hill_responses, 1e-6, 1)

    #         # log-sum-exp for product
    #         log_effects = interaction_matrix.T @ np.log(hill_clipped)
    #         product_effects = np.exp(log_effects)  # shape: (n_genes, num_cells)

    #         # Scale product by number of non-zero regulators per target gene
    #         regulator_counts = (interaction_matrix != 0).sum(axis=0).reshape(-1, 1)  # shape: (n_genes, 1)
    #         scaled_effects = regulator_counts * product_effects

    #     else:
    #         raise ValueError("EC50 must be a 1D array.")

    #     base_k_on = param_matrix['k_on'].values.reshape(n_genes, 1)
    #     k_on_adjusted = base_k_on + max_effect * scaled_effects
    # else:
    #     k_on_adjusted = np.tile(param_matrix['k_on'].values.reshape(n_genes, 1), (1, num_cells))

    # k_on_adjusted = np.maximum(k_on_adjusted, 1e-10)
    # new_state[:, s["k_on_adjusted"], :] = k_on_adjusted
    
    # === STEP 6: BURST SWITCHING - ALL GENES & CELLS AT ONCE ===
    
    # Current burst states
    is_bursting = new_state[:, s["is_bursting"], :].astype(bool)
    
    # k_off rates from DataFrame column
    # Switch probabilities for ALL genes and cells simultaneously
    switch_off_prob = np.random.exponential(1 / k_off_adjusted) < t
    switch_on_prob = np.random.exponential(1 / k_on_adjusted) < t
    
    # Apply switching logic - pure boolean operations on entire matrices
    should_switch_off = is_bursting & switch_off_prob
    should_switch_on = (~is_bursting) & switch_on_prob
    
    # Update burst states for entire matrix
    new_burst_state = (is_bursting | should_switch_on) & (~should_switch_off)
    new_state[:, s["is_bursting"], :] = new_burst_state.astype(float)
    
    # === FINAL: ENSURE NON-NEGATIVE VALUES ===
    new_state = np.maximum(new_state, 0)
    
    return new_state


In [9]:
def create_random_cells(n_genes, num_cells=1000, mean_protein_levels=None):
    """
    Create a random state array for a given number of genes and cells.
    
    Args:
        n_genes (int): Number of genes
        num_cells (int): Number of cells
    
    Returns:
        np.ndarray: Random state array of shape (n_genes, n_state_vars, num_cells)
    """
    state_vars_per_gene = ["is_bursting", "unspliced_mRNA", "spliced_mRNA", "protein",
                          "k_on_adjusted", "total_mRNA", "k_off_adjusted",
                          "mRNA_ever_produced", "protein_ever_produced"]
    
    state_var_indices = {var: i for i, var in enumerate(state_vars_per_gene)}
    
    # Initialize state array with zeros
    state_array = np.zeros((n_genes, len(state_vars_per_gene), num_cells))
    #Protein levels for each gene are between 0 and 3*mean_protein_levels for the gene
    if mean_protein_levels is None:
        mean_protein_levels = np.random.uniform(0, 100, n_genes)
    else:
        mean_protein_levels = np.asarray(mean_protein_levels).reshape(n_genes)

    # Randomly initialize protein levels
    state_array[:, state_var_indices["protein"], :] = np.random.uniform(
        0, 3 * mean_protein_levels[:, np.newaxis], size=(n_genes, num_cells)
    )
    print(state_array.shape)
    return state_array, state_var_indices

In [10]:

def select_parameter_rows(csv_path, rows=None, n_random=3):
    """
    Select parameter rows and return matrix format.
    
    Args:
        csv_path (str): Path to CSV file
        rows (list, optional): Specific row indices to select
        n_random (int): Number of random rows if rows=None
    
    Returns:
        tuple: (param_matrix, row_mapping)
    """
    df = pd.read_csv(csv_path, index_col=0)
    
    if rows is None:
        rows = np.random.choice(df.index, size=min(n_random, len(df)), replace=False)
    
    param_dict = {}
    row_mapping = {}
    
    for i, row in enumerate(rows):
        if row not in df.index:
            raise ValueError(f"Row {row} not found in the DataFrame.")
        gene_id = f"gene_{i+1}"
        param_dict[gene_id] = df.loc[row].to_dict()
        row_mapping[gene_id] = row
    
    # Create parameter matrix: genes as rows, parameters as columns
    param_matrix = pd.DataFrame.from_dict(param_dict, orient='index')
    
    return param_matrix, row_mapping


In [11]:
def simulate(path_to_matrix, parameter_sheet_path, num_cells, global_params= None, rows=None):
    
    if global_params is None:
        # Default global parameters if not provided
        global_params = {
            "splicing_rate": np.log(2)/(7/60),  # Splicing rate in per minute
            "max_effect": 100,  # Maximum effect for Hill function
            "n": 2,  # Hill coefficient
        }
    print("Reading input matrix and parameters...")
    n_genes, interaction_matrix = read_input_matrix(path_to_matrix)
    print(interaction_matrix)
    param_matrix, row_mapping = assign_parameters_to_genes(parameter_sheet_path, rows=rows, n_random=n_genes)

    #Calculate EC50 matrix
    print("Calculating EC50 matrix...")
    EC50_vector = create_EC50_vector(param_matrix, interaction_matrix, global_params)
    print(EC50_vector)
    #Burn in - not needed because I already have mean and burn-in is just giving me one more mean

    #Create random cells
    print("Creating random cells...")
    state_array, state_var_indices = create_random_cells(n_genes, num_cells=num_cells, mean_protein_levels=EC50_vector)
    #simulation time settings
    t = 1/60  # 1 minute time step
    total_time = 13 * 24 * 60 
    time_to_start_measurement = 12 * 24 * 60  # Start measuring after 12 days
    frequency_measurement = 60  # Measure every 60 minutes\
    steps = np.arange(0, total_time, int(t*60))
    division_time = 12 * 24 * 60  # Division occurs after 12 days
    pre_division_steps = np.arange(0, min(total_time, division_time), 1)
    after_division_steps = steps[steps > division_time] - division_time
    state_list = []
    replicate_list = []
    print("Starting simulation...")
    for curr_step in pre_division_steps:
        state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
    print("Simulation before division complete.")
    replicate_state = state_array.copy()
    state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=curr_step))
    state_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=curr_step))

    for curr_step in after_division_steps:
        state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
        replicate_state = get_new_state(replicate_state.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
        
        if curr_step >= time_to_start_measurement - division_time and curr_step % frequency_measurement == 0:
            state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=curr_step))
            replicate_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=curr_step))
    
    print("Simulation after division complete.")
    state_array_df = pd.concat(state_list, ignore_index=True)
    state_array_df['replicate'] = 0
    replicate_array_df = pd.concat(replicate_list, ignore_index=True)
    replicate_array_df['replicate'] = 1
    # Combine both dataframes
    combined_df = pd.concat([state_array_df, replicate_array_df], ignore_index=True)
    # Save the combined DataFrame to a CSV file

    return combined_df



In [12]:
# path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/matrix101.txt"
# path_to_param = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/parameter_sheet101.csv"
# num_cells = 10000
# rows = [0,0]
# df = simulate(path_to_matrix, path_to_param, num_cells, global_params= None, rows=rows)
# output_folder = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/"
# file_path = os.path.join(output_folder, "simulation_matrix101_A-|B_mod_kOff_max_effect_100.csv")
# df.to_csv(file_path, index=False)


In [16]:
import numpy as np
import os
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from tqdm_joblib import tqdm_joblib

path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/matrix101.txt"
path_to_param = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/parameter_sheet101.csv"
num_cells = 10000
rows = [0, 9]
output_folder = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/r_add_k_off_repression/"
os.makedirs(output_folder, exist_ok=True)
r_add_range = np.arange(40, 101, 1)
print(f"Total runs: {len(r_add_range)}")

def run_simulation_for_r_add(r_add):
    global_params = {
        "splicing_rate": np.log(2) / (7 / 60),
        "n": 2,
        "max_effect": r_add,
    }
    
    try:
        df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows)
        filename = f"simulation_matrix101_A-|B_mod_kOff_rAdd_{r_add:.2f}_n_2_min_k_off.csv"
        file_path = os.path.join(output_folder, filename)
        df.to_csv(file_path, index=False)
        return f"✅ r_add={r_add:.2f} done"
    except Exception as e:
        return f"❌ r_add={r_add:.2f} failed: {e}"

# Parallel execution with progress bar
with tqdm_joblib(tqdm(desc="Running simulations", total=len(r_add_range))) as _:
    results = Parallel(n_jobs=8)(
        delayed(run_simulation_for_r_add)(r_add) for r_add in r_add_range
    )

# Print summary
for res in results:
    print(res)


Total runs: 61


Running simulations:   0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Calculating EC50 matrix...
Calculating EC50 matrix...
Reading input matrix and parameters...
[68.74872801  2.54870309]
Creating random cells...
[68.74872801  2.48911271]
Creating random cells...
(2, 9, 10000)
Starting simulation...
[[ 0 -1]
 [ 0  0]]
(2, 9, 10000)
Starting simulation...
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Reading input matrix and parameters...
Calculating EC50 matrix...
[[ 0 -1]
 [ 0  0]]
[68.74872801  2.43224519]
Creating random cells...
(2, 9, 10000)
Starting simulation...
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Calculating EC50 matrix...
[68.74872801  2.37791808]
Creating random cells...
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]](2, 9, 10000)

Starting simulation...
Calculating EC50 matrix...
[68.74872801  2.32596486]
Creating random cells...
(2, 9, 10000)
Starting simulation...
Reading input matri

In [15]:
import numpy as np
import os
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from tqdm_joblib import tqdm_joblib

path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/matrix101.txt"
path_to_param = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/parameter_sheet101.csv"
num_cells = 10000
rows = [0, 0]
output_folder = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/r_add_k_off_repression/"
os.makedirs(output_folder, exist_ok=True)
r_add_range = np.arange(40, 101, 1)
print(f"Total runs: {len(r_add_range)}")

def run_simulation_for_r_add(r_add):
    global_params = {
        "splicing_rate": np.log(2) / (7 / 60),
        "n": 2,
        "max_effect": r_add,
    }
    
    try:
        df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows)
        filename = f"simulation_matrix101_A-|B_mod_kOff_rAdd_{r_add:.2f}_n_2.csv"
        file_path = os.path.join(output_folder, filename)
        df.to_csv(file_path, index=False)
        return f"✅ r_add={r_add:.2f} done"
    except Exception as e:
        return f"❌ r_add={r_add:.2f} failed: {e}"

# Parallel execution with progress bar
with tqdm_joblib(tqdm(desc="Running simulations", total=len(r_add_range))) as _:
    results = Parallel(n_jobs=8)(
        delayed(run_simulation_for_r_add)(r_add) for r_add in r_add_range
    )

# Print summary
for res in results:
    print(res)


Total runs: 61


Running simulations:   0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Calculating EC50 matrix...
Reading input matrix and parameters...
[68.74872801 12.24892365]
Creating random cells...
[[ 0 -1]
 [ 0  0]]
(2, 9, 10000)
Starting simulation...
Calculating EC50 matrix...
Reading input matrix and parameters...
[68.74872801 12.00232669]
Creating random cells...
[[ 0 -1]
 [ 0  0]]
(2, 9, 10000)
Starting simulation...
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
Calculating EC50 matrix...
[68.74872801 11.76546282]
Creating random cells...
Reading input matrix and parameters...
[[ 0 -1]
 [ 0  0]]
(2, 9, 10000)
Starting simulation...
Calculating EC50 matrix...
Calculating EC50 matrix...
[68.74872801 11.31871693]
Creating random cells...
Reading input matrix and parameters...
(2, 9, 10000)
Starting simulation...
[68.74872801 11.53776696]
Creating random cells...
[[ 0 -1]
 [ 0  0]]
(2, 9, 10000)
Starting simulation...
Calculating EC50 ma